In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 21
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)

556949
556928
['\ufeffThe Project Gutenber', 'The Project Gutenberg', 'he Project Gutenberg ', 'e Project Gutenberg E', ' Project Gutenberg EB']



torch.Size([556928, 21])


In [8]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(char2i, i2char)
net.to(device)
performance = net.fit(encoded, save_path = "checkpoints/autoregressive_lstm.pt")
net.save_architecture("architectures/autoregressive_lstm.architecture")

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

Epoch | Train                 | Training time
      | Loss     | Error Rate |
---------------------------------------------
    1 |   1.9706 |     50.012 |         47.92
    2 |   1.3346 |     37.500 |         96.02
    3 |   1.1832 |     33.895 |        144.07
    4 |   1.1067 |     32.094 |        192.33
    5 |   1.0584 |     30.944 |        240.65


In [9]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5])

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([-16.2640, -17.0789, -16.3670, -16.9348, -19.7253], device='cuda:0')


['\ufeffThe Project Gutenberg-tm the pawn at Kt5',
 'The Project Gutenberg-tm the pawn at Kt5 ',
 'he Project Gutenberg 1. ... K-Kt5; 12. Kt',
 'e Project Gutenberg E game in the pawn at',
 ' Project Gutenberg EB-K4 is a pawn at Kt5']

In [10]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5])

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([-34.2615, -41.1441, -29.4518, -43.6911, -49.2374], device='cuda:0')


['\ufeffThe Project Gutenberg all-hances, whight',
 'The Project GutenbergLobjertgatens.\n\n    ',
 'he Project Gutenberg 19L.\n\n\nBlack would o',
 'e Project Gutenberg Esmpult ic for with a',
 ' Project Gutenberg EBUsicl. after Yokk, B']

In [11]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5])

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([[ -6.1016,  -7.0423,  -8.4072,  -8.4236,  -9.2888],
        [ -5.9182,  -6.8601,  -8.2863,  -8.4591,  -9.1900],
        [ -7.7467,  -9.8321, -10.8760, -11.0262, -11.1016],
        [ -7.3991,  -8.5551,  -9.3477, -10.0063, -10.4871],
        [-14.4735, -14.6115, -14.7318, -14.9337, -14.9415]], device='cuda:0')
[['\ufeffThe Project Gutenberg\n\n        ---------',
  '\ufeffThe Project Gutenberg\n\n         --------',
  '\ufeffThe Project Gutenberg\n\n                 ',
  '\ufeffThe Project Gutenberg\n\n               Di',
  '\ufeffThe Project Gutenberg\n\n                D'],
 ['The Project Gutenberg\n\n        ----------',
  'The Project Gutenberg\n\n         ---------',
  'The Project Gutenberg\n\n               Dia',
  'The Project Gutenberg\n\n                  ',
  'The Project Gutenberg\n\

In [12]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
idx, log_probabilities = net.predict(encoded[:5])

net.tensor2text(idx)

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335



['\ufeffThe Project Gutenberg\n\n        ---------',
 'The Project Gutenberg\n\n        ----------',
 'he Project Gutenberg 2. .................',
 'e Project Gutenberg End-game.\n\n          ',
 ' Project Gutenberg EBlack cannot play the']